In [165]:
# %pip install --upgrade langchain langsmith langgraph langchain_openai

In [166]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OPENAI_API_KEY from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Multi Agent Collaboration"

### Set up the Tools

In [167]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda

# Set up the system template with a variable for context
system_template = """
Generate working code for a Jupyter Notebook based on the user's request. Your code should use LangChain, and specifically use LangChain's Expression Language in structuring your code.

Strictly adhere to the code examples delimited by triple backticks below as context for how LangChain's API works. DO NOT use any patterns that you do not find in the example below, unless you are 100% certain they work in LangChain:

```
{context}
```

Before sharing, double check your work. I will tip you $100 if your code is perfect.

Do not explain your work, just share working code.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Set up the human template with a variable for the request
human_template = """
{request}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
output_parser = StrOutputParser()

code_writing_runnable = prompt | model | output_parser

In [168]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

### Set up the agent

In [169]:
### Define the response schema for our agent
from langchain_core.pydantic_v1 import BaseModel, Field


class Response(BaseModel):
    """final answer to the user"""

    result: int = Field(description="the result of the computation")
    explanation: str = Field(
        description="explanation of the steps taken to get the result"
    )

In [170]:
### Set up the agent's tools

tools = []
pseudo_tools_visible = [
    "Retrieve Context",
    "Write Code",
    "Review Code",
    # "Save Code",
]
pseudo_tools_hidden = [
    "Store Request",
]

agent_tools = tools + pseudo_tools_visible + pseudo_tools_hidden

In [171]:
print(agent_tools)

['Retrieve Context', 'Write Code', 'Review Code', 'Store Request']


In [172]:
from langchain.pydantic_v1 import BaseModel
from enum import Enum
from langchain.tools import StructuredTool


# Set the agent options, which is FINISH plus all tools, with the exception of the hidden tools
agent_options = ["FINISH"] + agent_tools
agent_options = [item for item in agent_options if item not in pseudo_tools_hidden]

RouteOptions = Enum("RouteOptions", {option: option for option in agent_options})


class RouteInput(BaseModel):
    next: RouteOptions


def route(route: str) -> str:
    return route


router = StructuredTool.from_function(
    func=route,
    name="route",
    description="Select the next team member to use",
    args_schema=RouteInput,
    return_direct=False,
)

In [173]:
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
)

system_prompt_initial = """
You are a supervisor tasked with managing a development team consisting of the following members: {members}.

Given the following feature request from a user, respond with the worker to act next.

Each worker will perform a task and respond with their results and status. This task is complete as soon as you know a worker has retrieved context related to the user's feature request. When the task is complete, respond with FINISH.

You typically follow this pattern:

1) Retrieve context related to the user's query. This is a REQUIRED step before writing code
2) Write code to solve the problem
3) Save the code you have written once the reviewer has approved the code
"""

# Get the prompt to use - you can modify this!
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_prompt_initial),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
).partial(options=str(agent_options), members=", ".join(agent_tools))

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, [router], prompt)

### Set up the Agent State

In [174]:
from typing import TypedDict, Annotated, Sequence, Union
import operator
from langchain_core.messages import BaseMessage
from langchain_core.agents import AgentAction, AgentFinish

class AgentState(TypedDict):
    # The list of previous messages in the conversation
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The user's original request
    original_request: str
    # The input string
    # input: str
    # The outcome of a given call to the agent
    # Needs `None` as a valid type, since this is what this will start as
    agent_outcome: Union[AgentAction, AgentFinish, None]
    # List of actions and corresponding observations
    # Here we annotate this with `operator.add` to indicate that operations to
    # this state should be ADDED to the existing values (not overwrite it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]
    # The context
    context: str
    # The code
    code: str
    # The code quality
    code_approved: bool

### Set up the node actions

In [175]:
test = AgentState(intermediate_steps=[], messages=[], agent_outcome = None, original_request="hello", context="", code_approved=False, code="")
print(test)
agent_runnable.invoke(test)

{'intermediate_steps': [], 'messages': [], 'agent_outcome': None, 'original_request': 'hello', 'context': '', 'code_approved': False, 'code': ''}


AgentActionMessageLog(tool='route', tool_input={'next': 'Retrieve Context'}, log="\nInvoking: `route` with `{'next': 'Retrieve Context'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"next":"Retrieve Context"}', 'name': 'route'}})])

In [176]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage, AIMessage


# Define the function that determines whether to continue or not
def should_continue(state):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(state["agent_outcome"], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    # messages = state['messages']
    agent_outcome = agent_runnable.invoke(state)
    return {"agent_outcome": agent_outcome}


def call_set_initial_state(state):
    messages = state["messages"]
    last_message = messages[-1]
    return {"original_request": last_message.content}


# Define the function to execute tools
def call_tool(state):
    # We construct an ToolInvocation from the function_call
    tool = state['agent_outcome'].tool_input['next']
    print("Running Tool: ", tool)

    if tool == "Retrieve Context":
        print("Retreive Context")
        new_message = AIMessage(content="You have context now")
        return {"context": "You have context now", "messages": [new_message]}
    elif tool == "Write Code":
        print("successfully writing code now")
        code_writing_runnable.invoke({"context": "Some Context", "request": "My Request"})
        new_message = AIMessage(content="You have code now")
        return {"code": "You have context now", "messages": [new_message]}
        print("Write Code")
    elif tool == "Review Code":
        print("successfully reviewing code now")
        code_writing_runnable.invoke({"context": "Some Context", "request": "My Request"})
        new_message = AIMessage(content="Code is approved")
        return {"code_approved": True, "messages": [new_message]}
        print("Write Code")
    elif tool == "Save Code":
        print("Save Code")

    print("Slipped through the loop")
    return {"messages": [last_message]}

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )

    # response = input(prompt=f"[y/n] continue with: {action}?")
    # if response == "n":
    #     raise ValueError("User cancelled")
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

### Define the Graph

In [177]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.add_node("initial_state", call_set_initial_state)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("initial_state")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")
workflow.add_edge("initial_state", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

### Run our agent

In [178]:
from langchain_core.messages import HumanMessage

feature_request = """
Create a chain that does the following:
- Accept a string named answer as input
- Format a System and Human message using templates. The System message has output instructions via Pydantic. The Human message uses answer as context. Output instructions should require format to a Pydantic schema for hmw_question with a question (up to 10 words) and a role (either CMO, CTO, or CEO) 
- Pass the messages to OpenAI
- Parse the response using Pydantic
"""

inputs = {"messages": [HumanMessage(content=feature_request)]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'initial_state':
---
{'original_request': '\nCreate a chain that does the following:\n- Accept a string named answer as input\n- Format a System and Human message using templates. The System message has output instructions via Pydantic. The Human message uses answer as context. Output instructions should require format to a Pydantic schema for hmw_question with a question (up to 10 words) and a role (either CMO, CTO, or CEO) \n- Pass the messages to OpenAI\n- Parse the response using Pydantic\n'}

---

Output from node 'agent':
---
{'agent_outcome': AgentActionMessageLog(tool='route', tool_input={'next': 'Retrieve Context'}, log="\nInvoking: `route` with `{'next': 'Retrieve Context'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"next":"Retrieve Context"}', 'name': 'route'}})])}

---

Running Tool:  Retrieve Context
Retreive Context
Output from node 'action':
---
{'context': 'You have context now', 'messages': [AIMessag

In [179]:
import json
message_output = output['__end__']['messages']

for message in message_output:
    print(message)

content='\nCreate a chain that does the following:\n- Accept a string named answer as input\n- Format a System and Human message using templates. The System message has output instructions via Pydantic. The Human message uses answer as context. Output instructions should require format to a Pydantic schema for hmw_question with a question (up to 10 words) and a role (either CMO, CTO, or CEO) \n- Pass the messages to OpenAI\n- Parse the response using Pydantic\n'
content='You have context now'
content='You have code now'
content='Code is approved'


In [180]:
print(output['__end__']['messages'])

[HumanMessage(content='\nCreate a chain that does the following:\n- Accept a string named answer as input\n- Format a System and Human message using templates. The System message has output instructions via Pydantic. The Human message uses answer as context. Output instructions should require format to a Pydantic schema for hmw_question with a question (up to 10 words) and a role (either CMO, CTO, or CEO) \n- Pass the messages to OpenAI\n- Parse the response using Pydantic\n'), AIMessage(content='You have context now'), AIMessage(content='You have code now'), AIMessage(content='Code is approved')]


In [181]:
print(workflow)